In [415]:
!pip install gensim

In [416]:
import pandas as pd
import numpy as np
import gensim
import re
from tqdm import trange
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

In [417]:
df = pd.read_csv("D:\RecSys\products (1).csv")
for i in range(2,33):
    df2 = pd.read_csv("D:\RecSys\products (" + str(i) + ").csv")
    df = pd.concat([df,df2], ignore_index=True)

In [418]:
df.to_csv("D:\RecSys\products.csv")

In [479]:
df.sample(50)

,name,description,article
115,IQ color Бумага цветная яркая А4 80г/м2 500л ж...,Цветная бумага IQ COLOR интенсивный желтый неон,416493
514,Пазл детский в аммортименте Haio Xiang 100,NaN,121818
428,Lego 43189 Книга приключений Эльзы,NaN,111769
585,Деревяшка-каталка,NaN,651230
454,Пирамидка развивающая HAPPY SNAIL,NaN,114502
453,Набор игровой Маяк DEVIK,NaN,114478
102,"Цветная бумага IQ COLOR оранжевый неон, 80г 500л",Цветная бумага MAESTRO COLOR,314727
391,Lego 71385 Марио тануки,NaN,110617
100,IQ Color Бумага цветная яркая А4 80 г/м2 500 л...,Цветная бумага IQ COLOR интенсивный солнечно-ж...,416497
399,Lego 31113 Транспот гоночный авто,NaN,111773


плохие рекомендации если похожих товаров почти нет

In [510]:
class ContentBasedRec():
    def __init__(self, products, user_interactions):
        self.products = products
        self.interactions = np.unique(user_interactions)
        
        stop_words = stopwords.words('russian') + ['ассортимент', 'ассортименте', 'ассортим', 'см', 'м', 'шт', 'л', 'г']
        values = np.zeros(len(stop_words))
        self.stop_words_dict = dict(zip(stop_words, values))
        
        self.get_products_embeddings()
        
    def get_avg_vector(self, words, model, num_features, index_words_set):
        feature_vec = np.zeros((num_features, ), dtype='float32')
        n_words = 0
        for word in words:
            if word in index_words_set:
                n_words += 1
                feature_vec = np.add(feature_vec, model.wv[word])
        if (n_words > 0):
            feature_vec = feature_vec / n_words
        return feature_vec    
    
    def preprocess(self, text, stop_words_dict):
        word_list = []
        for word in re.findall("[а-яА-Яa-zA-ZёЁ]+", text.lower()):
            if word not in stop_words_dict:
                word_list.append(word)
        return word_list
    
    def get_products_embeddings(self):
        products_descriptions = []
        for i in trange(0,df['name'].shape[0]):
            products_descriptions.append(self.preprocess(df['name'].loc[i], self.stop_words_dict))
            
        self.model = gensim.models.Word2Vec(
                sentences=products_descriptions,
                vector_size=100,                  #размер эмбеддингов
                window=10,
                min_count=3)
                #iter=100)
        self.index_words_set = set(self.model.wv.index_to_key)
        self.embeddings = np.zeros((len(products_descriptions), 100))
        for i in range(self.embeddings.shape[0]):
            self.embeddings[i] = self.get_avg_vector(products_descriptions[i], self.model, 100, self.index_words_set)
            
    def get_rec_U2I(self):
        user_vector = " ".join(self.products[self.products['article']==_id]['name'].to_string() for _id in self.interactions)
        
        user_processed = self.preprocess(user_vector, self.stop_words_dict)
        user_emb = self.get_avg_vector(user_processed, self.model, 100, self.index_words_set)
        metrics = cosine_similarity([user_emb], self.embeddings)
        self.products['metrics']=metrics.reshape(-1)
        print(u"Для пользователя, который взаимодействовал с товарами\n")
        for id_ in self.interactions:
            print(self.products[self.products['article']==id_]['name'].to_string()+" {}".format(id_))
        print(u"\nТакие рекомендации")
        recomendations = pd.DataFrame(self.products.sort_values(by='metrics', ascending=False).loc[:,['name', 'metrics', 'article']])
        for product in self.interactions:
            recomendations.drop(recomendations[recomendations.article==product].index, inplace = True)
        print(recomendations[:15])

In [515]:
CB = ContentBasedRec(df.copy(), [66591, 67222])

100%|█████████████████████████████████████████████████████████████████████████████| 645/645 [00:00<00:00, 14022.46it/s]


In [516]:
vec = CB.get_rec_U2I()

Для пользователя, который взаимодействовал с товарами

354    Пластиковый конструктор Ми-ми-мишки 66591
545    Трансформер Tobot Атлон Торнадо 67222

Такие рекомендации
                                                  name   metrics  article
615                Дикие скричеры машин-трансформер л6  0.652510    73821
534              Трансформер - истребитель Taiko robot  0.566558    71864
530  Игрушка-трансформер TOBOT Atlon mini в ассорти...  0.498400   666302
386  Конструктор уход за животными на ферме LEGO 10949  0.476698   113058
441              Конструктор LEGO Mia's Pug Cube 41664  0.476698   110608
540                      Конструктор Blockformers 1TOY  0.449708   114490
350  Конструктор LEGO Friends 41426 Кафе в парке Ха...  0.418808    89483
364  Конструктор LEGO Duplo 10931 Грузовик и гусени...  0.393812    89523
431   Конструктор LEGO Friends 41427 Модный бутик Эммы  0.377994    89482
356  Конструктор LEGO Friends 41425 Цветочный сад О...  0.377994    89484
352         Конст